In [95]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sqlalchemy import create_engine

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [96]:
# postgres pull

db_password = "postgres"
db_user = "postgres"
db_name = "Youtube P3"
endpoint = 'youtube.cb1bticre0py.us-east-1.rds.amazonaws.com'

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

all_data = pd.read_sql('SELECT * FROM all_data;', con = engine)

In [97]:
# initial quick cleanup

all_data.fillna(all_data['dislikes'].median()) \
        .drop(all_data[all_data.video_id == '#NAME?'] \
        .index, inplace=True)

all_data.sort_values('trending_date', inplace = True)

all_data.rename(columns={'publishedat': 'publish_date', 
                         'channelid': 'channel_id', 
                         'channeltitle': 'channel_title', 
                         'likes':'likes_count', 
                         'dislikes': 'dislikes_count', 
                         'cat_codes': 'category'
                        }, inplace=True)

############################ trending day column must be added to all_data first
temp_df = all_data.groupby('video_id')['trending_date'].count().reset_index()
temp_df.rename({'trending_date':'trending_days'}, axis = 'columns', inplace = True)
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
############################

all_data.reset_index(inplace=True)

In [98]:
# changed publish_date & trending_date to datetime format

all_data[["publish_date", "trending_date"]] = all_data[["publish_date", "trending_date"]].apply(pd.to_datetime)

# add columns day_of_week and number_of_day

all_data['day_of_week'] = all_data['publish_date'].dt.day_name()
all_data['day_of_week_num'] = all_data['publish_date'].dt.day_of_week

# add column trending_days_by_country by subtracting one date from other (leaves in datetime format)

all_data['trending_days_by_country'] = all_data['trending_date'] - all_data['publish_date']

# add interactions ratios

all_data['likes_ratio'] = all_data['likes_count'] / all_data['view_count']*100
all_data['comments_ratio'] = all_data['comment_count'] / all_data['view_count']*100

# add publish_to_trend column

all_data['publish_to_trend'] = ((all_data['trending_date'] - all_data['publish_date']).dt.days).astype(float)

In [99]:
# temp_df = all_data[['likes_count', 'publish_to_trend']]
# temp_df['pt_likes_count_temp'] = temp_df['likes_count'] / temp_df['publish_to_trend']
# all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
# all_data

In [100]:
# temp_df = temp_df['likes_count'] / temp_df['publish_to_trend']
# temp_df

In [101]:
# temp_df.rename({'trending_date':'trending_days'}, axis = 'columns', inplace = True)

In [102]:
# all_data dataframe start

print(all_data.shape)
print("              ")
print(all_data.dtypes)
print("              ")
all_data.head(2)

(1300294, 20)
              
index                                 int64
video_id                             object
publish_date                 datetime64[ns]
channel_id                           object
channel_title                        object
trending_date                datetime64[ns]
view_count                            int64
likes_count                         float64
dislikes_count                      float64
comment_count                       float64
thumbnail_link                       object
country                              object
category                             object
trending_days                         int64
day_of_week                          object
day_of_week_num                       int64
trending_days_by_country    timedelta64[ns]
likes_ratio                         float64
comments_ratio                      float64
publish_to_trend                    float64
dtype: object
              


,index,video_id,publish_date,channel_id,channel_title,trending_date,view_count,likes_count,dislikes_count,comment_count,thumbnail_link,country,category,trending_days,day_of_week,day_of_week_num,trending_days_by_country,likes_ratio,comments_ratio,publish_to_trend
0,0,J78aPJ3VyNs,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,2020-08-12,2038853,353790.0,2628.0,40228.0,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,United Kingdom,Entertainment,22,Tuesday,1,1 days,17.352404,1.973070,1.0
1,1,J78aPJ3VyNs,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,2020-08-12,2038853,353794.0,2628.0,40222.0,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,Germany,Entertainment,22,Tuesday,1,1 days,17.352600,1.972776,1.0


In [103]:
# unique dataframe start

unique = all_data.drop_duplicates(subset = 'video_id', keep='last')

print(unique.shape)
print("              ")
print(unique.dtypes)
print("              ")
unique.head(2)

(241113, 20)
              
index                                 int64
video_id                             object
publish_date                 datetime64[ns]
channel_id                           object
channel_title                        object
trending_date                datetime64[ns]
view_count                            int64
likes_count                         float64
dislikes_count                      float64
comment_count                       float64
thumbnail_link                       object
country                              object
category                             object
trending_days                         int64
day_of_week                          object
day_of_week_num                       int64
trending_days_by_country    timedelta64[ns]
likes_ratio                         float64
comments_ratio                      float64
publish_to_trend                    float64
dtype: object
              


,index,video_id,publish_date,channel_id,channel_title,trending_date,view_count,likes_count,dislikes_count,comment_count,thumbnail_link,country,category,trending_days,day_of_week,day_of_week_num,trending_days_by_country,likes_ratio,comments_ratio,publish_to_trend
21,21,J78aPJ3VyNs,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,2020-08-18,3490530,457130.0,4269.0,47291.0,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,United States,Entertainment,22,Tuesday,1,7 days,13.096292,1.354837,7.0
28,28,HkTTPu_p2ps,2020-08-08,UC-yXuc1__OzjwpsJPlxYUCQ,WORLDSTARHIPHOP,2020-08-14,628065,35101.0,1158.0,1992.0,https://i.ytimg.com/vi/HkTTPu_p2ps/default.jpg,Canada,Music,7,Saturday,5,6 days,5.588753,0.317165,6.0


In [109]:
y = unique.groupby('video_id')['trending_days'].sum()
y.rename({'trending_days':'pt_trending_days'}, axis = 'columns', inplace = True)

In [110]:
y

video_id
#NAME?         1387
--0bCF-iK2E       5
--14w5SOEUs      51
--2O86Z0hsM       9
--2Us2lFR8Y       6
               ... 
zzoZr-dNlks       6
zztGoiavb5c      11
zzuXBF16ZQg       2
zzx2oiznzyc       2
zzxPZwaA-8w       1
Name: trending_days, Length: 241113, dtype: int64